# <span style="color:#900C3F;">Mice Protein Expression Data Analysis</span>

### <span style="color:#5E5C5B;">In this notebook, using mice protein expression data, we construct frequent pattern mining models and clustering analysis models, and implement them above on the data, and analyze the performance of each classification model.</span> 

### <span style="color:#900C3F;">Steps taken:</span> 

1. **Setting up the environment.**
2. **Preparing Data for Analysis.**
3. **Implementation and Model evaluation.**
4. **Comparing the performance of classification models.**

## <span style="color:#900C3F;">1. Setting up the environment</span>

**Importing requried libraries**

Mainly used:
- **Apache spark** in this project, so we import the some libraries required, **pyspark** for starting a **spark session.** 
- **pandas** for data analysis and manipulation.
- **sklearn** for statistical modeling including classification.
- **plotly** for visualization.

In [21]:
import pyspark
import pandas as pd
import numpy as np
import pprint
from sklearn import metrics, svm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import plotly.graph_objects as go
import plotly.express as px

## <span style="color:#900C3F;">2. Preparing Data for Analysis</span>

**Preparing** dataset taken from http://archive.ics.uci.edu/ml/index.php for the analysis.




In [22]:
#reading the file with pandas's method read_excel, converting the csv file to a dataframe
def readCSVtoDF(csvData):
    return pd.read_excel(csvData)

rawData = 'data/Data_Cortex_Nuclear.xls'

DF = readCSVtoDF(rawData)

In [23]:
DF

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
1,309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
2,309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
3,309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
4,309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,J3295_11,0.254860,0.463591,0.254860,2.092082,2.600035,0.211736,0.171262,2.483740,0.207317,...,0.183324,0.374088,0.318782,0.204660,0.328327,1.364823,Ts65Dn,Saline,S/C,t-SC-s
1076,J3295_12,0.272198,0.474163,0.251638,2.161390,2.801492,0.251274,0.182496,2.512737,0.216339,...,0.175674,0.375259,0.325639,0.200415,0.293435,1.364478,Ts65Dn,Saline,S/C,t-SC-s
1077,J3295_13,0.228700,0.395179,0.234118,1.733184,2.220852,0.220665,0.161435,1.989723,0.185164,...,0.158296,0.422121,0.321306,0.229193,0.355213,1.430825,Ts65Dn,Saline,S/C,t-SC-s
1078,J3295_14,0.221242,0.412894,0.243974,1.876347,2.384088,0.208897,0.173623,2.086028,0.192044,...,0.196296,0.397676,0.335936,0.251317,0.365353,1.404031,Ts65Dn,Saline,S/C,t-SC-s


In [24]:
DF['MouseNumber'] = DF.MouseID.apply(lambda x: x.split('_')[0]) 
DF['MeasurementNumber'] = DF.MouseID.apply(lambda x: x.split('_')[1])
DF.drop(['MouseID'], axis=1, inplace=True)  

In [25]:
DF

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,MouseNumber,MeasurementNumber
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m,309,1
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m,309,2
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m,309,3
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m,309,4
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m,309,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.254860,0.463591,0.254860,2.092082,2.600035,0.211736,0.171262,2.483740,0.207317,1.057971,...,0.318782,0.204660,0.328327,1.364823,Ts65Dn,Saline,S/C,t-SC-s,J3295,11
1076,0.272198,0.474163,0.251638,2.161390,2.801492,0.251274,0.182496,2.512737,0.216339,1.081150,...,0.325639,0.200415,0.293435,1.364478,Ts65Dn,Saline,S/C,t-SC-s,J3295,12
1077,0.228700,0.395179,0.234118,1.733184,2.220852,0.220665,0.161435,1.989723,0.185164,0.884342,...,0.321306,0.229193,0.355213,1.430825,Ts65Dn,Saline,S/C,t-SC-s,J3295,13
1078,0.221242,0.412894,0.243974,1.876347,2.384088,0.208897,0.173623,2.086028,0.192044,0.922595,...,0.335936,0.251317,0.365353,1.404031,Ts65Dn,Saline,S/C,t-SC-s,J3295,14


### <span>Dataset</span>

**Number of instances and Number of attributes**

In [26]:
DF.shape

(1080, 83)

**Preprocessing data**

In [45]:
isNullGenotype = len(DF[pd.isnull(DF["Genotype"])])
isNullTreatment = len(DF[pd.isnull(DF["Treatment"]) ]) 
isNullBehavior = len(DF[pd.isnull(DF["Behavior"])]) 
print(isNullGenotype, isNullTreatment, isNullBehavior)

0 0 0


In [13]:
DF['BCL2_N'][:20] #to check if there is null values in BCL2_N column

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15    0.178604
16    0.143588
17    0.132887
18    0.182391
19    0.163775
Name: BCL2_N, dtype: float64

In [14]:
#replacing missing values with mean values of each column
meanValues = DF.groupby(by=['class']).mean().reset_index()  #mean values of numerical attributes

classValue = 'class'
def handleNullValues(row, column):
    value = row[column]
    if pd.isnull(value):
        return meanValues.loc[meanValues[classValue] == row[classValue], column], True
    return value, False

for i, row in DF.iterrows():  #series for each row
    for column in DF.columns:
        row[column], handled = handleNullValues(row, column)
    DF.iloc[i] = row

In [16]:
DF['BCL2_N'][:20] # to check if null values are filled by mean value in BCL2_N column

0     0.132539
1     0.132539
2     0.132539
3     0.132539
4     0.132539
5     0.132539
6     0.132539
7     0.132539
8     0.132539
9     0.132539
10    0.132539
11    0.132539
12    0.132539
13    0.132539
14    0.132539
15    0.178604
16    0.143588
17    0.132887
18    0.182391
19    0.163775
Name: BCL2_N, dtype: float64

In [47]:
def minMaxNormalization(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns[:77]:  #for numeric values
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm
    
# call the min_max_scaling function
normalizedDF = minMaxNormalization(DF)

normalizedDF

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,MouseNumber,MeasurementNumber
0,0.151122,0.212885,0.824638,0.612119,0.630482,0.327006,0.448666,0.168257,0.617322,0.232553,...,0.087715,0.102890,0.084580,0.705738,Control,Memantine,C/S,c-CS-m,309,1
1,0.155750,0.188226,0.776455,0.601070,0.585247,0.311887,0.429899,0.154925,0.590173,0.205362,...,0.080692,0.115874,0.093977,0.749771,Control,Memantine,C/S,c-CS-m,309,2
2,0.153459,0.205696,0.793572,0.558911,0.575910,0.306369,0.441381,0.153485,0.607102,0.199194,...,0.080465,0.109050,0.082162,0.868229,Control,Memantine,C/S,c-CS-m,309,3
3,0.125169,0.157688,0.637326,0.468152,0.480646,0.335530,0.444307,0.132074,0.486945,0.205135,...,0.126763,0.164241,0.144543,0.721879,Control,Memantine,C/S,c-CS-m,309,4
4,0.122146,0.157838,0.637787,0.426467,0.441977,0.314976,0.433100,0.129086,0.410194,0.189152,...,0.096959,0.136298,0.149281,0.812053,Control,Memantine,C/S,c-CS-m,309,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.046197,0.092577,0.365672,0.313684,0.127872,0.312096,0.423753,0.186230,0.488562,0.110645,...,0.597619,0.388482,0.725820,0.504334,Ts65Dn,Saline,S/C,t-SC-s,J3295,11
1076,0.053509,0.097062,0.357235,0.342243,0.157739,0.395191,0.468154,0.190968,0.535204,0.114722,...,0.614760,0.371843,0.614028,0.504110,Ts65Dn,Saline,S/C,t-SC-s,J3295,12
1077,0.035163,0.063556,0.311370,0.165795,0.071655,0.330863,0.384915,0.105509,0.374040,0.080099,...,0.603927,0.484631,0.811962,0.547100,Ts65Dn,Saline,S/C,t-SC-s,J3295,13
1078,0.032018,0.071071,0.337173,0.224787,0.095856,0.306129,0.433086,0.121245,0.409605,0.086829,...,0.640497,0.571336,0.844450,0.529739,Ts65Dn,Saline,S/C,t-SC-s,J3295,14


## <span style="color:#900C3F;">3. Implementation and Model evaluation</span>

### <span style="color:#900C3F;">Frequent Pattern Mining</span>

#### <span>Apriori</span>

#### <span>FP-Growth</span>

#### <span>ECLAT</span>

### <span style="color:#900C3F;">Clustering Analysis</span>

#### <span>K-Means</span>

#### <span>AGNES</span>

#### <span>DBSCAN</span>

## <span style="color:#900C3F;">4. Comparing the performance of classification models</span>

## Conclusion

#### References: 